## UFO Sightings K-Means Clustering
### Modeling Lab

The goal of this notebook is to analyze where Mr. K should build his extraterrestrial life facilities using the K-Means algorithm. 

What we plan on accomplishing is the following:
1. [Load dataset onto Notebook instance from S3](#Step-1:-Loading-the-data-from-Amazon-S3)
2. [Cleaning, transforming, and preparing the data](#Step-2:-Cleaning,-transforming,-and-preparing-the-data)
3. [Create and train our model](#Step-3:-Create-and-train-our-model)
4. [Viewing the results](#Step-4:-Viewing-the-results)
5. [Visualize using QuickSight](https://docs.aws.amazon.com/quicksight/latest/user/create-a-data-set-s3.html)

First let's go ahead and import all the needed libraries.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Step 1: Loading the data from Amazon S3
Next, lets get the UFO sightings data that is stored in S3.

In [2]:
role = get_execution_role()
bucket = 'modeling-ufo-lab1'
prefix = 'ufo_dataset'
data_key = 'ufo_fullset.csv'
data_location = 's3://{}/{}/{}'.format(bucket, prefix, data_key)

df = pd.read_csv(data_location, low_memory=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [3]:
df.head()

,reportedTimestamp,eventDate,eventTime,shape,duration,witnesses,weather,firstName,lastName,latitude,longitude,sighting,physicalEvidence,contact,researchOutcome
0,1977-04-04T04:02:23.340Z,1977-03-31,23:46,circle,4,1,rain,Ila,Bashirian,47.329444,-122.578889,Y,N,N,explained
1,1982-11-22T02:06:32.019Z,1982-11-15,22:04,disk,4,1,partly cloudy,Eriberto,Runolfsson,52.664913,-1.034894,Y,Y,N,explained
2,1992-12-07T19:06:52.482Z,1992-12-07,19:01,circle,49,1,clear,Miller,Watsica,38.951667,-92.333889,Y,N,N,explained
3,2011-02-24T21:06:34.898Z,2011-02-21,20:56,disk,13,1,partly cloudy,Clifton,Bechtelar,41.496944,-71.367778,Y,N,N,explained
4,1991-03-09T16:18:45.501Z,1991-03-09,11:42,circle,17,1,mostly cloudy,Jayda,Ebert,47.606389,-122.330833,Y,N,N,explained


In [4]:
df.shape

(18000, 15)

## Step 2: Cleaning, transforming, and preparing the data
Create another DataFrame with just the latitude and longitude attributes

In [5]:
df_geo = df[['latitude', 'longitude']]

In [6]:
df_geo.head()

,latitude,longitude
0,47.329444,-122.578889
1,52.664913,-1.034894
2,38.951667,-92.333889
3,41.496944,-71.367778
4,47.606389,-122.330833


In [7]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   18000 non-null  float64
 1   longitude  18000 non-null  float64
dtypes: float64(2)
memory usage: 281.4 KB


In [8]:
missing_values = df_geo.isnull().values.any()
print('Are there any missing values? {}'.format(missing_values))
if(missing_values):
    df_geo[df_geo.isnull().any(axis=1)]

Are there any missing values? False


Next, let's go ahead and transform the pandas DataFrame (our dataset) into a numpy.ndarray. When we do this each row is converted to a Record object. According to the documentation, this is what the K-Means algorithm expects as training data. This is what we will use as training data for our model.

[See the documentation for input training](https://sagemaker.readthedocs.io/en/stable/kmeans.html)

In [9]:
data_train = df_geo.values.astype('float32')
data_train

array([[  47.329445, -122.57889 ],
       [  52.664913,   -1.034894],
       [  38.951668,  -92.333885],
       ...,
       [  36.86639 ,  -83.888885],
       [  35.385834,  -94.39833 ],
       [  29.883055,  -97.94111 ]], dtype=float32)

## Step 3: Create and train our model
In this step we will import and use the built-in SageMaker K-Means algorithm. We will set the number of cluster to 10 (for our 10 sensors), specify the instance type we want to train on, and the location of where we want our model artifact to live. 

[See the documentation of hyperparameters here](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means-api-config.html)

In [10]:
from sagemaker import KMeans

num_clusters = 10
output_location = 's3://' + bucket + '/model-artifacts'

kmeans = KMeans(role=role,
               instance_count=1,
               instance_type='ml.c4.xlarge',
               output_path=output_location,
               k=num_clusters)

In [11]:
job_name = 'kmeans-geo-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))

Here is the job name kmeans-geo-job-20240624172337


In [12]:
%%time
kmeans.fit(kmeans.record_set(data_train), job_name=job_name)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: kmeans-geo-job-20240624172337


2024-06-24 17:23:39 Starting - Starting the training job...
2024-06-24 17:23:54 Starting - Preparing the instances for training...
2024-06-24 17:24:25 Downloading - Downloading input data...
2024-06-24 17:24:55 Downloading - Downloading the training image.........
2024-06-24 17:26:37 Training - Training image download completed. Training in progress.
2024-06-24 17:26:37 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[06/24/2024 17:26:29 INFO 139987154560832] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_

## Step 4: Viewing the results
In this step we will take a look at the model artifact SageMaker created for us and stored onto S3. We have to do a few special things to see the latitude and longitude for our 10 clusters (and the center points of those clusters)

[See the documentation of deserilization here](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#td-deserialization)

At this point we need to "deserilize" the model artifact. Here we are going to open and review them in our notebook instance. We can unzip the model artifact which will contain model_algo-1. This is just a serialized Apache MXNet object. From here we can load that serialized object into a numpy.ndarray and then extract the clustered centroids from the numpy.ndarray.

After we extract the results into a DataFrame of latitudes and longitudes, we can create a CSV with that data, load it onto S3 and then visualize it with QuickSight.

In [13]:
import os
model_key = 'model-artifacts/' + job_name + '/output/model.tar.gz'

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

model_algo-1
Archive:  model_algo-1


tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of model_algo-1 or
        model_algo-1.zip, and cannot find model_algo-1.ZIP, period.


2304

In [14]:
!pip install mxnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.5 MB/s eta 0:00:0000:01:00:01


In [15]:
import mxnet as mx
Kmeans_model_params = mx.ndarray.load('model_algo-1')

In [31]:
Kmeans_model_params

[
 [[  35.336853   -98.741165 ]
  [  49.87383     -3.7976685]
  [  -4.999058   112.205666 ]
  [  34.935562  -118.7064   ]
  [  31.74403    -82.60426  ]
  [  41.340214   -75.29815  ]
  [  46.147224  -119.76389  ]
  [  62.17071   -148.79977  ]
  [  40.93326    -87.650185 ]
  [   1.0341873  -67.69947  ]]
 <NDArray 10x2 @cpu(0)>]

In [32]:
cluster_centroids_kmeans = pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids_kmeans.columns=df_geo.columns
cluster_centroids_kmeans

,latitude,longitude
0,35.336853,-98.741165
1,49.873829,-3.797668
2,-4.999058,112.205666
3,34.935562,-118.706398
4,31.744030,-82.604263
5,41.340214,-75.298149
6,46.147224,-119.763893
7,62.170712,-148.799774
8,40.933262,-87.650185
9,1.034187,-67.699471


Let's go ahead and upload this dataset onto S3 and view within QuickSight

In [33]:
from io import StringIO

csv_buffer = StringIO()
cluster_centroids_kmeans.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'results/ten_locations_kmeans.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5XBKAJEHDKMH74DN',
  'HostId': '9MAxQv+sh+EsK0mOnzJ7JK5Im8MUKW7QfsCUt+R6JcfKSFQih75kqdiRwVnuVnBDKoSe6dmOGw4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9MAxQv+sh+EsK0mOnzJ7JK5Im8MUKW7QfsCUt+R6JcfKSFQih75kqdiRwVnuVnBDKoSe6dmOGw4=',
   'x-amz-request-id': '5XBKAJEHDKMH74DN',
   'date': 'Mon, 24 Jun 2024 19:01:45 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"fa689843dacbd5531082ec596ec96765"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fa689843dacbd5531082ec596ec96765"',
 'ServerSideEncryption': 'AES256'}

In [34]:
!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 16.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 27.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00:00:01


Import geopandas and other libraries

In [35]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt

INFO:matplotlib.font_manager:generated new fontManager


In [36]:
# Getting world map data from geo pandas
worldmap = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))

# Creating axes and plotting world map
fig, ax = plt.subplots(figsize=(16, 10))
worldmap.plot(color="lightgrey", ax=ax)

AttributeError: The geopandas.dataset has been deprecated and was removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.